#### Build a single-turn RAG Pipeline, that answers questions from a documentation-style web-page, using the correct splitter and chain setup.

#### Setting up the environment

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

#### 1) Loading the data

In [48]:
from langchain_community.document_loaders import WebBaseLoader
import bs4 

loader = WebBaseLoader(
    'https://docs.langchain.com/oss/python/langchain/rag?utm_source=chatgpt.com',
    bs_kwargs=dict(parse_only = bs4.SoupStrainer(
        id = 'content-container'
    )
    ) # This helps scrape only the main content
)

In [49]:
doc = loader.load()

In [50]:
print('Loaded Docs:', len(doc))

Loaded Docs: 1


In [52]:
print("Sample chars:", len(doc[0].page_content))
print("Sample preview:\n", doc[0].page_content[:2500])

Sample chars: 30691
Sample preview:
 On this pageOverviewConceptsPreviewSetupInstallationLangSmithComponents1. IndexingLoading documentsSplitting documentsStoring documents2. Retrieval and generationRAG agentsRAG chainsNext stepsTutorialsLangChainBuild a RAG agent with LangChainCopy pageCopy page​Overview
One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.
This tutorial will show how to build a simple Q&A application over an unstructured text data source. We will demonstrate:

A RAG agent that executes searches with a simple tool. This is a good general-purpose implementation.
A two-step RAG chain that uses just a single LLM call per query. This is a fast and effective method for simple queries.

​Concepts
We will cover the following concepts:


Indexing: a pipeline f

#### 2) Splitting the text into chunks

In [54]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 200
)
chunks = splitter.split_documents(doc)

print('\n Total chunks:', len(chunks))
print('One Chunk preview:', chunks[0].page_content[:500])


 Total chunks: 41
One Chunk preview: On this pageOverviewConceptsPreviewSetupInstallationLangSmithComponents1. IndexingLoading documentsSplitting documentsStoring documents2. Retrieval and generationRAG agentsRAG chainsNext stepsTutorialsLangChainBuild a RAG agent with LangChainCopy pageCopy page​Overview
One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known


#### 3) Creating Embeddings for the chunks

In [55]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings() 

#### 4) Creating vectore Store (FAISS)

In [56]:
from langchain_classic.vectorstores import FAISS

vector_db = FAISS.from_documents(chunks, embeddings)

print('Embedding dimension (from FAISS:)', vector_db.index.d)

Embedding dimension (from FAISS:) 1536


#### 5) Retriever 

In [57]:
retriever = vector_db.as_retriever(search_kwargs ={'k': 4})

#### 6) Document Chain (LLM + Prompt)

In [61]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model = 'gpt-4o')

prompt = ChatPromptTemplate.from_template(
    """
You are a helpful assistant.
Answer the following using ONLY the context below.

<context>
{context}
</context>

Question: {input}

Rules:
- If the answer is not in the context, say: I do not know based on the provided context.
-Keep the answer short and direct.
"""
)

In [63]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [64]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant.\nAnswer the following using ONLY the context below.\n\n<context>\n{context}\n</context>\n\nQuestion: {input}\n\nRules:\n- If the answer is not in the context, say: I do not know based on the provided context.\n-Keep the answer short and direct.\n'), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'stru

#### 7) Retriever Chain

In [65]:
from langchain_classic.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [66]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1484ce900>, search_kwargs={'k': 4}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant.\nAnswer the following using ONLY the context below.\n\n<context>\n{context}\n</context>\n\nQuestion: 

#### 8) Testing with a few queries

In [74]:
questions = [
    'What are the main stages of a RAG pipeline described in the tutorial?',
    "What is LangChain's revenue in 2025?"
]


for q in questions:
    print('\n' + '='*90)
    print('Question:', q)
    resu = retrieval_chain.invoke({'input': q})

    print('\nAnswer:\n', resu['answer'])

    print('\n Retrieved Contact:')
    for i, d in enumerate(resu['context'], 1):
        preview = d.page_content[:500].replace('\n', " ")
        print(f'\n Chunk {i} preview: {preview} ....')
        print(' Metadata:', d.metadata)


Question: What are the main stages of a RAG pipeline described in the tutorial?

Answer:
 The main stages of a RAG pipeline described in the tutorial are Indexing and Retrieval and Generation.

 Retrieved Contact:

 Chunk 1 preview: ​Concepts We will cover the following concepts:   Indexing: a pipeline for ingesting data from a source and indexing it. This usually happens in a separate process.   Retrieval and generation: the actual RAG process, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model. ....
 Metadata: {'source': 'https://docs.langchain.com/oss/python/langchain/rag?utm_source=chatgpt.com'}

 Chunk 2 preview: We can see the full sequence of steps, along with latency and other metadata, in the LangSmith trace. You can add a deeper level of control and customization using the LangGraph framework directly— for example, you can add steps to grade document relevance and rewrite search queries. Check out LangGraph’s 

In [72]:
questions = [
    'What are the main stages of a RAG pipeline described in the tutorial?',
    "What is LangChain's revenue in 2025?"
]


for q in questions:
    print('\n' + '='*90)
    print('Question:', q)
    resu = retrieval_chain.invoke({'input': q})

    print('\nAnswer:\n', resu['answer'])




Question: What are the main stages of a RAG pipeline described in the tutorial?

Answer:
 The main stages of a RAG pipeline described in the tutorial are indexing and retrieval and generation.

Question: What is LangChain's revenue in 2025?

Answer:
 I do not know based on the provided context.
